In [ ]:
# Import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as skl
import tensorflow as tf
import os
import cv2
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Define base directory
base_dir = '/content/drive/MyDrive/BOOTCAMP/ColabNotebooks/ProjectWithGreg/Data'

# Verify each directory level
drive_dir = '/content/drive/MyDrive'
bootcamp_dir = os.path.join(drive_dir, 'BOOTCAMP')
colab_notebooks_dir = os.path.join(bootcamp_dir, 'ColabNotebooks')
project_dir = os.path.join(colab_notebooks_dir, 'ProjectWithGreg')
data_dir = os.path.join(project_dir, 'Data')

Mounted at /content/drive


In [ ]:
# Use os.path.join() to concatenate base_dir and 'train', set train_dir to 'Data/train'
# os.path.join(base_dir, 'train') takes base_dir, which = 'Data/', and concatenates it with string 'train'
train_dir = os.path.join(base_dir, 'train')

# Concatenate base_dir and 'test', set test_dir to 'Data/test'
# os.path.join(base_dir, 'test') takes base_dir, which = 'Data/', and concatenates it with string 'test'
test_dir = os.path.join(base_dir, 'test')

# Concatenate base_dir and 'valid', set valid_dir to 'Data/valid'
# os.path.join(base_dir, 'valid') takes base_dir, which = 'Data/', and concatenates it with string 'valid'
valid_dir = os.path.join(base_dir, 'valid')

#Read contents of base_dir directory and return list of names of entries (files and directories) in it
os.listdir(base_dir)

['valid', 'test', 'train']

In [ ]:
#Data generators are a convenient way to load and preprocess data in batches during model training

#ImageDataGenerator: This class from Keras's ImageDataGenerator module is used to generate batches of tensor image data with real-time data augmentation

# Define data generators for training and validation
# train_datagen and valid_datagen are instances of ImageDataGenerator used for training and validation data
# The rescale=1./255 parameter scales pixel values to the range [0,1]
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

img_size = (224, 224)  # Define target size for images
batch_size = 32

# train_gen and valid_gen are actual data generators created using flow_from_directory method
# They load images from specified directories, rescale them, and convert labels to categorical format

# flow_from_directory method generates batches of augmented/normalized data from image files in a directory
# flow_from_directory method allows you to specify various parameters like target size, batch size, and class mode

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_size[0], img_size[1]),
    batch_size=batch_size,
    class_mode='categorical'
)

valid_gen = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_size[0], img_size[1]),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 613 images belonging to 4 classes.
Found 72 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.optimizers import Adam

# Assuming train_gen and valid_gen are your image data generators

img_size = (224, 224)
channels = 3
img_shape = (img_size[0], img_size[1], channels)
class_count = len(list(train_gen.class_indices.keys()))

# Create pre-trained ResNet50 model
base_model = ResNet50(include_top=False, weights="imagenet", input_shape=img_shape, pooling='max')
for layer in base_model.layers:
    layer.trainable = False
    model = Sequential([
    base_model,
    BatchNormalization(axis=-1),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(class_count, activation='softmax')
])

optimizer = Adam()  # Using Adam optimizer
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Define filepath to save the best model
filepath = 'best_model_resnet.h5'

# Create ModelCheckpoint callback to save the best model based on validation accuracy
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

# Train the model with the added callback
history = model.fit(
    x=train_gen,
    epochs=50,
    verbose=1,
    validation_data=valid_gen,
    callbacks=[checkpoint]
)

Epoch 1/100
20/20 [==============================] - ETA: 0s - loss: 1.1421 - accuracy: 0.5840 
Epoch 1: val_accuracy improved from -inf to 0.47222, saving model to best_model_resnet.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


20/20 [==============================] - 343s 17s/step - loss: 1.1421 - accuracy: 0.5840 - val_loss: 1.2137 - val_accuracy: 0.4722
Epoch 2/100
20/20 [==============================] - ETA: 0s - loss: 0.5943 - accuracy: 0.7537
Epoch 2: val_accuracy improved from 0.47222 to 0.51389, saving model to best_model_resnet.h5
20/20 [==============================] - 143s 7s/step - loss: 0.5943 - accuracy: 0.7537 - val_loss: 1.4465 - val_accuracy: 0.5139
Epoch 3/100
20/20 [==============================] - ETA: 0s - loss: 0.4282 - accuracy: 0.8189
Epoch 3: val_accuracy did not improve from 0.51389
20/20 [==============================] - 151s 8s/step - loss: 0.4282 - accuracy: 0.8189 - val_loss: 1.7187 - val_accuracy: 0.4861
Epoch 4/100
20/20 [==============================] - ETA: 0s - loss: 0.3680 - accuracy: 0.8679
Epoch 4: val_accuracy did not improve from 0.51389
20/20 [==============================] - 147s 7s/step - loss: 0.3680 - accuracy: 0.8679 - val_loss: 1.5791 - val_accuracy: 0.4861

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.layers import Input, Average

model= load_model('best_model_resnet.h5')

ts_length = len(test_df)
test_batch_size = test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))
test_steps = ts_length // test_batch_size
train_score = model.evaluate(train_gen, steps= test_steps, verbose= 1)
valid_score = model.evaluate(valid_gen, steps= test_steps, verbose= 1)
test_score = model.evaluate(test_gen, steps= test_steps, verbose= 1)

print("Train Loss: ", train_score[0])
print("Train Accuracy: ", train_score[1])
print('-' * 20)
print("Validation Loss: ", valid_score[0])
print("Validation Accuracy: ", valid_score[1])
print('-' * 20)
print("Test Loss: ", test_score[0])
print("Test Accuracy: ", test_score[1])